<a href="https://colab.research.google.com/github/Ifthekhar1342/New-Project/blob/main/Copy_of_Image_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Libaries

In [ ]:
# Importing all the necessary libraries
import keras
import h5py
from keras import optimizers
from keras.models import load_model
from keras.layers import Bidirectional
from keras.layers.core import Reshape, Dropout
from keras.utils.vis_utils import plot_model
import os
# import keras_metrics
import matplotlib.pyplot as plt
from keras.layers import Conv1D,Dense, MaxPooling1D, Flatten, GlobalAveragePooling2D,GlobalAveragePooling3D
from keras import regularizers
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from keras import regularizers
from keras.applications.inception_v3 import InceptionV3
from tensorflow.keras import models
from tensorflow.keras import layers
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import re
from nltk.corpus import stopwords
from nltk import word_tokenize
from keras.preprocessing import image
from PIL import Image, ImageFile
from keras import preprocessing, Input
import numpy as np
#model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
from keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.resnet50 import ResNet50
from keras.models import Model
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.metrics import classification_report,f1_score
import pickle as pkl

In [ ]:
# Keras Functional API

import tensorflow as tf
from tensorflow import keras
from keras.layers import Input, Dense, Activation, Dropout,Flatten,Embedding
from keras.layers import Conv1D,MaxPooling1D,GlobalAveragePooling1D
from keras.models import Model
from tensorflow.keras.optimizers import Adam,SGD,Nadam,RMSprop
from tensorflow.keras.models import load_model

# Keras Functional API

import tensorflow as tf
from tensorflow import keras
from keras.layers import Input, Dense, Activation, Dropout,Flatten,Embedding
from keras.layers import Conv1D,MaxPooling1D,GlobalAveragePooling1D, Bidirectional, LSTM, GRU
from keras.models import Model
from tensorflow.keras.models import load_model


In [ ]:
dataset_path = "/content/drive/MyDrive/DATASET/"
# Validation_path = "/content/drive/MyDrive/DATASET/val_data.csv"
img_dir = "/content/drive/MyDrive/multimodal_image"

## Fetching Data

In [ ]:
train_data = pd.read_excel(dataset_path+'train.xlsx')
test_data = pd.read_csv(dataset_path+'test.xlsx')
print("training size:", len(train_data))
print("testing size:", len(test_data))

training size: 5247
testing size: 584


In [ ]:
train_data.head()

,filename,tweet,label,image
0,buildingfire_2017-02-05_04-06-10.txt,Here's some video of the smoldering ruins in W...,damaged_infrastructure,buildingfire_2017-02-05_04-06-10.JPG
1,isiscrimes_2015-08-04_00-18-33.txt,27 person were killed yesterday near idlib nor...,human_damage,isiscrimes_2015-08-04_00-18-33.JPG
2,earthquake_2017-11-13_07-23-10.txt,تصویری از خسارات زلزله در سرپل ذهاب\n#earthqua...,damaged_infrastructure,earthquake_2017-11-13_07-23-10.JPG
3,ad_2017-11-25_07-56-53.txt,Link in bio #fashion #style #dt #friends #Inst...,non_damage,ad_2017-11-25_07-56-53.JPG
4,hurricanesandy_2017-10-29_22-36-42.txt,Covering the aftermath of Hurricane #Sandy 5 y...,flood,hurricanesandy_2017-10-29_22-36-42.JPG


In [ ]:
train_data.columns

Index(['filename', 'tweet', 'label', 'image'], dtype='object')

In [ ]:
train_data['label'].value_counts()

non_damage                2666
damaged_infrastructure    1246
damaged_nature             459
flood                      348
fires                      309
human_damage               219
Name: label, dtype: int64

In [ ]:
# Encode Labels
train_data["enc_label"]=train_data.label.replace({"non_damage":0,"damaged_infrastructure":1,"damaged_nature":2,"flood":3,"fires":4,"human_damage":5})
test_data["enc_label"]=test_data.label.replace({"non_damage":0,"damaged_infrastructure":1,"damaged_nature":2,"flood":3,"fires":4,"human_damage":5})

In [ ]:
# convert float type data into str if any occurs
train_data["tweet"]=train_data["tweet"].astype(str)
test_data["tweet"]=test_data["tweet"].astype(str)

In [ ]:
train_data['enc_label'].value_counts()

0    2666
1    1246
2     459
3     348
4     309
5     219
Name: enc_label, dtype: int64

In [ ]:
def replace_string(row):
  return row.replace('.JPG','.jpg')

train_data['image'] = train_data['image'].apply(replace_string)
test_data['image'] = test_data['image'].apply(replace_string)

## Reading Image

In [ ]:
def create_img_path(DF, Col_name, img_dir):
    img_path = [img_dir + '/' + name for name in DF[Col_name]]
    return img_path

In [ ]:
# Creating train, test and validation image path
train_img_path = create_img_path(train_data,'image', img_dir)
test_img_path = create_img_path(test_data,'image', img_dir)

In [ ]:
train_img_path[0]

'/content/drive/MyDrive/multimodal_image/buildingfire_2017-02-05_04-06-10.JPG'

#Image Fetching

In [ ]:
def get_input(path):
    ImageFile.LOAD_TRUNCATED_IMAGES = True
    img = Keras.utilis.load_img(path,target_size = (100,100))
    return(img)

In [ ]:
# Takes in image and preprocess it
def process_input(img):
    # Converting image to array
    img_data = keras.utils.img_to_array(img)
    # Adding one more dimension to array
    img_data = np.expand_dims(img_data, axis=0)
    #
    # img_data = preprocess_input(img_data)
    return(img_data)

In [ ]:
    from tensorflow.keras.preprocessing.image import load_img
    from tensorflow.keras.preprocessing.image import img_to_array
    from tensorflow.keras.applications.vgg16 import preprocess_input
    from tensorflow.keras.applications.vgg16 import decode_predictions
    from tensorflow.keras.applications.vgg16 import VGG16

In [ ]:
# Create an array of training images
train_images =[]
for n,i in enumerate(train_img_path):
      input_img = get_input(i)
      process_img = process_input(input_img)
      print(n)
      train_images.append(process_img[0])



NameError: ignored

In [ ]:
#  #Create an array of test images
 test_images = []
 for n,i in enumerate(test_img_path):
 input_img = get_input(i)
  process_img = process_input(input_img)
  print(n)
   test_images.append(process_img[0])


IndentationError: ignored

In [ ]:
#  convert into numpy array
 train_images  =  np.array(train_images)
# # convert into numpy array
 test_image = np.array(test_images)


IndentationError: ignored

In [ ]:
with open(dataset_path+'train.pkl','wb') as f:
    pkl.dump(train_image, f)

with open(dataset_path+'test.pkl','wb') as f:
    pkl.dump(test_image, f)

NameError: ignored

#Load Images

In [ ]:
with open(dataset_path+'train.pkl','rb') as f:
  train_image = pkl.load(f)
  print("Training Images:-- ",train_image.shape)

with open(dataset_path+'test.pkl','rb') as f:
  test_image = pkl.load(f)
  print("Test Images:-- ",test_image.shape)


EOFError: ignored

## Callback

In [ ]:
def checkpoint_fn(name):
  filepath =f'/content/drive/MyDrive/New folder/Model/{name}.h5'
  checkpoint = ModelCheckpoint(filepath=filepath, monitor='val_accuracy', verbose=1, save_best_only=True, save_weights_only=False, mode='auto')
  early = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=20, verbose=1, mode='auto')
  return [checkpoint,early]


## VGG16 model

In [ ]:
def VGG16_model(input_shape, n_classes, optimizer, fine_tune=0):
    """
    Compiles a model integrated with VGG16 pretrained layers

    input_shape: tuple - the shape of input images (width, height, channels)
    n_classes: int - number of classes for the output layer
    optimizer: string - instantiated optimizer to use for training. Defaults to 'RMSProp'
    fine_tune: int - The number of pre-trained layers to unfreeze.
                If set to 0, all pretrained layers will freeze during training
    """

    # Pretrained convolutional layers are loaded using the Imagenet weights.
    # Include_top is set to False, in order to exclude the model's fully-connected layers.
    conv_base = VGG16(include_top=False,
                     weights='imagenet',
                     input_shape=input_shape)

    # Defines how many layers to freeze during training.
    # Layers in the convolutional base are switched from trainable to non-trainable
    # depending on the size of the fine-tuning parameter.
    if fine_tune > 0:
        for layer in conv_base.layers[:-fine_tune]:
            layer.trainable = False
    else:
        for layer in conv_base.layers:
            layer.trainable = False

    # Create a new 'top' of the model (i.e. fully-connected layers).
    # This is 'bootstrapping' a new top_model onto the pretrained layers.
    top_model = conv_base.output
    top_model = Flatten()(top_model)
    # top_model = Dense(4096, activation='relu')(top_model)
    top_model = Dense(536, activation='relu')(top_model)
    # top_model = Dropout(0.2)(top_model)
    output_layer = Dense(n_classes, activation='softmax')(top_model)

    # Group the convolutional base and new fully-connected layers into a Model object.
    model = Model(inputs=conv_base.input, outputs=output_layer)

    # Compiles the model for training.
    model.compile(optimizer=optimizer,
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    return model

In [ ]:
input_shape = (100, 100, 3)
n_epochs = 30
n_classes = 6

In [ ]:
# Building img_prediction layer using Image_model
vgg16_img = VGG16_model(input_shape, n_classes,Adam(3e-3), fine_tune=2)

58889256/58889256 [==============================] - 0s 0us/step


In [ ]:
# base_img.summary()

In [ ]:
vgg16_history = vgg16_img.fit(train_image,train_data['enc_label'],
                            batch_size=32,
                            epochs=n_epochs,
                            callbacks=checkpoint_fn('vgg16'),
                            validation_split = 0.1)

Epoch 1/30
148/148 [==============================] - ETA: 0s - loss: 1.1508 - accuracy: 0.5726
Epoch 1: val_accuracy improved from -inf to 0.57333, saving model to /content/drive/MyDrive/New folder/Model/vgg16.h5
148/148 [==============================] - 627s 4s/step - loss: 1.1508 - accuracy: 0.5726 - val_loss: 1.1217 - val_accuracy: 0.5733
Epoch 2/30
148/148 [==============================] - ETA: 0s - loss: 1.1304 - accuracy: 0.5961
Epoch 2: val_accuracy improved from 0.57333 to 0.58667, saving model to /content/drive/MyDrive/New folder/Model/vgg16.h5
148/148 [==============================] - 636s 4s/step - loss: 1.1304 - accuracy: 0.5961 - val_loss: 1.1312 - val_accuracy: 0.5867
Epoch 3/30
148/148 [==============================] - ETA: 0s - loss: 1.1065 - accuracy: 0.6099
Epoch 3: val_accuracy did not improve from 0.58667
148/148 [==============================] - 623s 4s/step - loss: 1.1065 - accuracy: 0.6099 - val_loss: 1.1247 - val_accuracy: 0.5733
Epoch 4/30
148/148 [======

In [ ]:
from keras.models import load_model
vgg16_model = load_model('/content/drive/MyDrive/New folder/Model/vgg16.h5')

In [ ]:
y_pred=np.argmax(vgg16_model.predict(test_image),axis=-1)

19/19 [==============================] - 1s 60ms/step


In [ ]:
print(classification_report(test_data["enc_label"],y_pred))

              precision    recall  f1-score   support

           0       0.78      0.86      0.82       291
           1       0.74      0.57      0.64       144
           2       0.52      0.42      0.46        55
           3       0.35      0.61      0.45        36
           4       0.58      0.51      0.54        37
           5       0.55      0.29      0.37        21

    accuracy                           0.69       584
   macro avg       0.59      0.54      0.55       584
weighted avg       0.70      0.69      0.69       584



In [ ]:
f1_score(test_data["enc_label"], y_pred, average='weighted')

0.6852908675145871

# Build VGG19 model

In [ ]:
# from keras.applications.vgg16 import preprocess_input


## VGG19 Model

In [ ]:
def VGG19_model(input_shape, n_classes, optimizer='rmsprop'):
    """
    Compiles a model integrated with VGG16 pretrained layers

    input_shape: tuple - the shape of input images (width, height, channels)
    n_classes: int - number of classes for the output layer
    optimizer: string - instantiated optimizer to use for training. Defaults to 'RMSProp'
    fine_tune: int - The number of pre-trained layers to unfreeze.
                If set to 0, all pretrained layers will freeze during training
    """
    keras.backend.clear_session()
    # Pretrained convolutional layers are loaded using the Imagenet weights.
    # Include_top is set to False, in order to exclude the model's fully-connected layers.
    conv_base = VGG19(include_top=False,
                     weights='imagenet',
                     input_shape=input_shape)

    # Defines how many layers to freeze during training.
    # Layers in the convolutional base are switched from trainable to non-trainable
    # depending on the size of the fine-tuning parameter.

    for layer in conv_base.layers:
          layer.trainable = False

    # Create a new 'top' of the model (i.e. fully-connected layers).
    # This is 'bootstrapping' a new top_model onto the pretrained layers.
    top_model = conv_base.output
    top_model = Flatten()(top_model)
    # top_model = Dense(4096, activation='relu')(top_model)
    top_model = Dense(536, activation='relu')(top_model)
    # top_model = Dropout(0.2)(top_model)
    output_layer = Dense(n_classes, activation='softmax')(top_model)

    # Group the convolutional base and new fully-connected layers into a Model object.
    model = Model(inputs=conv_base.input, outputs=output_layer)

    # Compiles the model for training.
    model.compile(optimizer=optimizer,
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    return model

In [ ]:
input_shape = (100, 100, 3)
n_epochs = 30
n_classes = 6

In [ ]:
# Building img_prediction layer using Image_model
vgg19_img = VGG19_model(input_shape, n_classes,Adam(learning_rate=1e-3))

80134624/80134624 [==============================] - 0s 0us/step


In [ ]:
vgg19_history = vgg19_img.fit(train_image,train_data['enc_label'],
                            batch_size=32,
                           epochs=n_epochs,
                           callbacks=checkpoint_fn('vgg19'),
                            validation_split= 0.1 )

Epoch 1/30
147/148 [============================>.] - ETA: 0s - loss: 3.8976 - accuracy: 0.6420
Epoch 1: val_accuracy improved from -inf to 0.64571, saving model to /content/drive/MyDrive/New folder/Model/vgg19.h5
148/148 [==============================] - 11s 68ms/step - loss: 3.8874 - accuracy: 0.6425 - val_loss: 1.0972 - val_accuracy: 0.6457
Epoch 2/30
148/148 [==============================] - ETA: 0s - loss: 0.4281 - accuracy: 0.8431
Epoch 2: val_accuracy improved from 0.64571 to 0.71238, saving model to /content/drive/MyDrive/New folder/Model/vgg19.h5
148/148 [==============================] - 9s 58ms/step - loss: 0.4281 - accuracy: 0.8431 - val_loss: 1.1569 - val_accuracy: 0.7124
Epoch 3/30
148/148 [==============================] - ETA: 0s - loss: 0.1910 - accuracy: 0.9337
Epoch 3: val_accuracy did not improve from 0.71238
148/148 [==============================] - 8s 51ms/step - loss: 0.1910 - accuracy: 0.9337 - val_loss: 1.3516 - val_accuracy: 0.6952
Epoch 4/30
148/148 [=====

In [ ]:
from keras.models import load_model
saved_model = load_model('/content/drive/MyDrive/New folder/Model/vgg19.h5')

In [ ]:
y_pred=np.argmax(saved_model.predict(test_image),axis=-1)

19/19 [==============================] - 1s 45ms/step


In [ ]:
print(classification_report(test_data["enc_label"],y_pred))

              precision    recall  f1-score   support

           0       0.78      0.84      0.81       291
           1       0.64      0.67      0.66       144
           2       0.51      0.44      0.47        55
           3       0.58      0.53      0.55        36
           4       0.70      0.51      0.59        37
           5       0.71      0.48      0.57        21

    accuracy                           0.71       584
   macro avg       0.65      0.58      0.61       584
weighted avg       0.70      0.71      0.70       584



In [ ]:
f1_score(test_data["enc_label"], y_pred, average='weighted')

0.7019679040817086

# Build Inception Model

In [ ]:
# from keras.applications.inception_v3 import preprocess_input

## Inception Model

In [ ]:
def INCEPTION_model(input_shape, n_classes, optimizer='rmsprop'):
    """
    Compiles a model integrated with VGG16 pretrained layers

    input_shape: tuple - the shape of input images (width, height, channels)
    n_classes: int - number of classes for the output layer
    optimizer: string - instantiated optimizer to use for training. Defaults to 'RMSProp'
    fine_tune: int - The number of pre-trained layers to unfreeze.
                If set to 0, all pretrained layers will freeze during training
    """
    keras.backend.clear_session()
    # Pretrained convolutional layers are loaded using the Imagenet weights.
    # Include_top is set to False, in order to exclude the model's fully-connected layers.
    conv_base = InceptionV3(include_top=False,
                     weights='imagenet',
                     input_shape=input_shape)

    # Defines how many layers to freeze during training.
    # Layers in the convolutional base are switched from trainable to non-trainable
    # depending on the size of the fine-tuning parameter.

    for layer in conv_base.layers:
          layer.trainable = False

    # Create a new 'top' of the model (i.e. fully-connected layers).
    # This is 'bootstrapping' a new top_model onto the pretrained layers.
    top_model = conv_base.output
    top_model = Flatten()(top_model)
    # top_model = Dense(4096, activation='relu')(top_model)
    top_model = Dense(536, activation='relu')(top_model)
    # top_model = Dropout(0.2)(top_model)
    output_layer = Dense(n_classes, activation='softmax')(top_model)

    # Group the convolutional base and new fully-connected layers into a Model object.
    model = Model(inputs=conv_base.input, outputs=output_layer)

    # Compiles the model for training.
    model.compile(optimizer=optimizer,
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    return model

In [ ]:
input_shape = (100, 100, 3)
n_epochs = 30
n_classes = 6

In [ ]:
# Building img_prediction layer using Image_model
inception_img = INCEPTION_model(input_shape, n_classes,Adam(learning_rate=1e-3))

In [ ]:
incept_history = inception_img.fit(train_image,train_data['enc_label'],
                            batch_size=32,
                           epochs=n_epochs,
                           callbacks=checkpoint_fn('Inception'),
                            validation_split = 0.1)

Epoch 1/30
146/148 [============================>.] - ETA: 0s - loss: 19.8056 - accuracy: 0.3902
Epoch 1: val_accuracy improved from -inf to 0.36571, saving model to /content/drive/MyDrive/New folder/Model/Inception.h5
148/148 [==============================] - 10s 42ms/step - loss: 19.6558 - accuracy: 0.3914 - val_loss: 4.6393 - val_accuracy: 0.3657
Epoch 2/30
147/148 [============================>.] - ETA: 0s - loss: 3.5306 - accuracy: 0.4324
Epoch 2: val_accuracy improved from 0.36571 to 0.49333, saving model to /content/drive/MyDrive/New folder/Model/Inception.h5
148/148 [==============================] - 4s 30ms/step - loss: 3.5245 - accuracy: 0.4333 - val_loss: 3.3863 - val_accuracy: 0.4933
Epoch 3/30
146/148 [============================>.] - ETA: 0s - loss: 1.8597 - accuracy: 0.4912
Epoch 3: val_accuracy did not improve from 0.49333
148/148 [==============================] - 4s 24ms/step - loss: 1.8555 - accuracy: 0.4915 - val_loss: 1.7249 - val_accuracy: 0.4362
Epoch 4/30
147/

In [ ]:
from keras.models import load_model
saved_model = load_model('/content/drive/MyDrive/New folder/Model/Inception.h5')

In [ ]:
y_pred=np.argmax(saved_model.predict(test_image),axis=-1)

19/19 [==============================] - 2s 59ms/step


In [ ]:
print(classification_report(test_data["enc_label"],y_pred))

              precision    recall  f1-score   support

           0       0.56      0.88      0.69       291
           1       0.50      0.35      0.42       144
           2       0.33      0.04      0.07        55
           3       0.29      0.06      0.09        36
           4       0.29      0.05      0.09        37
           5       0.00      0.00      0.00        21

    accuracy                           0.54       584
   macro avg       0.33      0.23      0.23       584
weighted avg       0.47      0.54      0.46       584



In [ ]:
f1_score(test_data["enc_label"], y_pred, average='weighted')

0.463649643562885

# Build Resnet Model

In [ ]:

# from tensorflow.keras.applications.resnet50  import preprocess_input

## Resnet50

In [ ]:
def Resnet50_model(input_shape, n_classes, optimizer='rmsprop'):
    """
    Compiles a model integrated with VGG16 pretrained layers

    input_shape: tuple - the shape of input images (width, height, channels)
    n_classes: int - number of classes for the output layer
    optimizer: string - instantiated optimizer to use for training. Defaults to 'RMSProp'
    fine_tune: int - The number of pre-trained layers to unfreeze.
                If set to 0, all pretrained layers will freeze during training
    """

    # Pretrained convolutional layers are loaded using the Imagenet weights.
    # Include_top is set to False, in order to exclude the model's fully-connected layers.
    keras.backend.clear_session()
    conv_base = ResNet50(include_top=False,
                     weights='imagenet',
                     input_shape=input_shape)

    # Defines how many layers to freeze during training.
    # Layers in the convolutional base are switched from trainable to non-trainable
    # depending on the size of the fine-tuning parameter.
    for layer in conv_base.layers:
        layer.trainable = False

    # Create a new 'top' of the model (i.e. fully-connected layers).
    # This is 'bootstrapping' a new top_model onto the pretrained layers.
    top_model = conv_base.output
    top_model = GlobalAveragePooling2D()(top_model)
    # # top_model = Dense(4096, activation='relu')(top_model)
    top_model = Dense(100, activation='relu')(top_model)
    # top_model = Dropout(0.2)(top_model)
    output_layer = Dense(n_classes, activation='softmax')(top_model)

    # Group the convolutional base and new fully-connected layers into a Model object.
    model = Model(inputs=conv_base.input, outputs=output_layer)

    # Compiles the model for training.
    model.compile(optimizer=optimizer,
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    return model

In [ ]:
input_shape = (100, 100, 3)
n_epochs = 30
n_classes = 6

In [ ]:
# Building img_prediction layer using Image_model
resnet_img = Resnet50_model(input_shape, n_classes, Adam(learning_rate=1e-3))

94765736/94765736 [==============================] - 1s 0us/step


In [ ]:
resnet_history = resnet_img.fit(train_image,train_data['enc_label'],
                            batch_size=32,
                           epochs=n_epochs,
                           callbacks=checkpoint_fn('Resnet50'),
                            validation_split=0.1)

Epoch 1/30
148/148 [==============================] - ETA: 0s - loss: 0.9117 - accuracy: 0.6747
Epoch 1: val_accuracy improved from -inf to 0.71238, saving model to /content/drive/MyDrive/New folder/Model/Resnet50.h5
148/148 [==============================] - 19s 61ms/step - loss: 0.9117 - accuracy: 0.6747 - val_loss: 0.8575 - val_accuracy: 0.7124
Epoch 2/30
147/148 [============================>.] - ETA: 0s - loss: 0.6245 - accuracy: 0.7700
Epoch 2: val_accuracy did not improve from 0.71238
148/148 [==============================] - 6s 38ms/step - loss: 0.6230 - accuracy: 0.7706 - val_loss: 0.8873 - val_accuracy: 0.6857
Epoch 3/30
147/148 [============================>.] - ETA: 0s - loss: 0.5124 - accuracy: 0.8046
Epoch 3: val_accuracy did not improve from 0.71238
148/148 [==============================] - 5s 36ms/step - loss: 0.5127 - accuracy: 0.8045 - val_loss: 0.8178 - val_accuracy: 0.7029
Epoch 4/30
147/148 [============================>.] - ETA: 0s - loss: 0.4159 - accuracy: 0.8

In [ ]:
from keras.models import load_model
saved_model = load_model('/content/drive/MyDrive/New folder/Model/Resnet50.h5')

In [ ]:
y_pred=np.argmax(saved_model.predict(test_image),axis=-1)

19/19 [==============================] - 2s 52ms/step


In [ ]:
print(classification_report(test_data["enc_label"],y_pred))

              precision    recall  f1-score   support

           0       0.82      0.80      0.81       291
           1       0.70      0.61      0.65       144
           2       0.43      0.58      0.49        55
           3       0.68      0.69      0.68        36
           4       0.63      0.70      0.67        37
           5       0.60      0.57      0.59        21

    accuracy                           0.71       584
   macro avg       0.64      0.66      0.65       584
weighted avg       0.72      0.71      0.72       584



In [ ]:
f1_score(test_data["enc_label"], y_pred, average='weighted')

0.7173581334519564

# Text Model

## Clean Text

In [ ]:
import nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
STOPWORDS = set(stopwords.words('english'))
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
EMAIL = re.compile('^([a-zA-Z0-9_\-\.]+)@([a-zA-Z0-9_\-\.]+)\.([a-zA-Z]{2,5})$')
# NUMBERS = re.compile(['0-9'])
STOPWORDS = set(stopwords.words('english'))

In [ ]:
# def clean_text(text):
#     """
#         text: a string

#         return: modified initial string
#     """
#     text = text.lower()
#     text = EMAIL.sub('', text)
# # #     text = NUMBERS.sub('',text)
# #     text = REPLACE_BY_SPACE_RE.sub(' ',text)
# #     text = BAD_SYMBOLS_RE.sub('',text)
# #     text = text.replace('x','')
#     # text = ' '.join(word for word in text.split() if word not in STOPWORDS)

#     return text

In [ ]:
'''
Text Cleaning
'''
from bs4 import BeautifulSoup
def clean_text(row):
  #to remove HTML tags
  text = BeautifulSoup(row, 'html.parser').get_text()
  d = re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b', '', text, flags=re.MULTILINE) #This line is for removing url
  post = d.replace('\n', '')
  post = post.replace('—', ' ')
  # to remove accented characters
  # to remove special characters and numbers
  # define the pattern to keep
  pat = r'[^#@a-zA-z0-9]'
  text = re.sub(pat, ' ', post)
  #to remove punctuation
  #text = ''.join([c for c in text if c not in string.punctuation])
  # to remove special characters
  #pattern = r'^\s*|\s\s*'
  #text = re.sub(pattern, ' ', text).strip()
  # convert into lower case
  text = text.lower()
  # Stopword Removing
  #tokenizer = ToktokTokenizer()
  # convert sentence into token of words
  #tokens = tokenizer.tokenize(text)
  #tokens = [token.strip() for token in tokens]


  return text

In [ ]:
train_data['clean_Text'] = train_data['tweet'].apply(clean_text)
test_data['clean_Text'] = test_data['tweet'].apply(clean_text)

In [ ]:
test_data['clean_Text'][0]

'  we are really getting into the christmas spirit at miracle tiny toes    this little miracle and her friends are getting excited for christmas watch to find out why   featuring kendal leigh #baby #babyboutique #babies1stchristmas #babiesofinstagram #babykeepsake #keepsake #christmas #winter #instagram #babyshower #babygift #unisex #babyboy #babygirl #perfect #fashion #fashionista #advert #ad #advertising #festive #boutiquestore #christmasshopping #shop #miracles #miracletinytoes #worldwideshipping #worldwide #organic #instavideo'

In [ ]:
np.unique(train_data['enc_label'])

array([0, 1, 2, 3, 4, 5])

In [ ]:
train_data['enc_label'].value_counts()

0    2666
1    1246
2     459
3     348
4     309
5     219
Name: enc_label, dtype: int64

In [ ]:
test_data.head()

,filename,tweet,label,image,enc_label,clean_Text
0,ad_2017-11-25_10-36-26.txt,⭐ We are really getting into the christmas spi...,non_damage,ad_2017-11-25_10-36-26.jpg,0,we are really getting into the christmas spi...
1,building_2017-10-30_17-26-34.txt,IJOY uv board has the competetive price and ve...,non_damage,building_2017-10-30_17-26-34.jpg,0,ijoy uv board has the competetive price and ve...
2,floodwater_2017-09-04_04-46-10.txt,Arriving in Kalkundi island destroyed in #bang...,flood,floodwater_2017-09-04_04-46-10.jpg,3,arriving in kalkundi island destroyed in #bang...
3,accrafloods_2015-06-06_16-59-56.txt,"Hi my lovelies, check out My first hand exper...",flood,accrafloods_2015-06-06_16-59-56.jpg,3,hi my lovelies check out my first hand exper...
4,buildingfire_2016-10-02_03-07-17.txt,The Hamilton fire service during an exercise a...,fires,buildingfire_2016-10-02_03-07-17.jpg,4,the hamilton fire service during an exercise a...


## Tokenization

In [ ]:
%%time
max_words = 100000

from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words = max_words, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n-',
                      split=' ', char_level=False, oov_token=None, document_count=0)

tokenizer.fit_on_texts(train_data['tweet'])


CPU times: user 313 ms, sys: 16.3 ms, total: 329 ms
Wall time: 330 ms


In [ ]:
%%time
word_counts = tokenizer.word_counts
word_docs = tokenizer.word_docs
word_index = tokenizer.word_index
document_count = tokenizer.document_count
vocab_size = len(word_counts) + 1
print(vocab_size)

44857
CPU times: user 1.14 ms, sys: 0 ns, total: 1.14 ms
Wall time: 1.12 ms


# Encoding Data into Numbers

In [ ]:
%%time
# Convert string into list of integer indices
train_sequences = tokenizer.texts_to_sequences(train_data['tweet'])
test_sequences = tokenizer.texts_to_sequences(test_data['tweet'])

CPU times: user 234 ms, sys: 0 ns, total: 234 ms
Wall time: 234 ms


In [ ]:
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))
# print(train_data['cleanText'])
print(train_sequences[4663])


Found 44856 unique tokens.
[1008, 187, 3238, 15, 100, 41618, 3721, 1057, 1008, 325, 41619]


### Pad sequence

In [ ]:
%%time
## Before padding length of different training examples
# train data
mx = len(train_sequences[0])
for x in train_sequences:
  mx =max(mx, len(x))
print("Maximum Text length",mx)
#test data
mx = len(test_sequences[0])
for x in test_sequences:
  mx =max(mx, len(x))
print("Maximum Text length",mx)
#validation data
mx = len(test_sequences[0])
for x in test_sequences:
  mx =max(mx, len(x))
print("Maximum Text length",mx)

Maximum Text length 386
Maximum Text length 415
Maximum Text length 415
CPU times: user 8.1 ms, sys: 0 ns, total: 8.1 ms
Wall time: 8.13 ms


In [ ]:
from keras_preprocessing.sequence import pad_sequences

In [ ]:
##We can also determine maxlen by plotting the frequency distribution of the lengths
train_corpus = pad_sequences(train_sequences, value=0.0, padding='post', maxlen= 100)
test_corpus = pad_sequences(test_sequences, value=0.0, padding='post', maxlen= 100)

## We get the maxlen value from the Length frequency distribution

print(train_corpus[10])
print(test_corpus[1])



[15779   966   617     3    13   415    20     4    51   778  1470   167
    20    15  6758 15780  3372   126    14     1   778  1470     9  1086
    19   132  1471     3   388    35   252   618   629   582  1818     2
   132   925   451    62    12   710     2  9443    11   110  9444  2274
  9445    11  3033     3    15  1941     2     9   342     1   159     2
   287  6759   211     3  3373    11  3033    10 15781 15782  9444 15783
     7     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0]
[17046  2459    54     1   879     2   252   110   350  2675  5955    79
    42 17046  2459 17046  2459   266 17102 17046 10668   397   266  2459
 17046  8265  3722 17046   266 17102 15707 17106  2459  1141  2873   737
    17  5295  1238  5622  1238  8670   875    95   137  1861  1141  5704
  5704 31841  8248     0     0     0     0     0     0     0     0     0
     0     0     0     0 

In [ ]:
y_train=np.array(train_data['enc_label']).reshape(-1,1)
y_test=np.array(test_data['enc_label']).reshape(-1,1)

## CNN with keras embedding

## Model Parameters

In [ ]:
max_length = 100
embedding_dim = 100
number_of_classes = 6

In [ ]:
keras.backend.clear_session()

# define CNN model

def CNN():

  input = Input(shape=(max_length,))
  embedding = Embedding(max_words, embedding_dim)(input)
  conv1 = Conv1D(128,2,activation='relu')(embedding)
  pool1 = MaxPooling1D(2)(conv1)
  flat = Flatten()(pool1)
  output_layer = Dense(number_of_classes, activation='softmax')(flat)
  model = Model(inputs=input, outputs=output_layer)

  return model

# call the model
cnn_model = CNN()

# cnn_model.summary()


In [ ]:
cnn_model.compile(optimizer = Adam(),
                      loss = 'sparse_categorical_crossentropy',
                      metrics =['accuracy'])

In [ ]:
%%time
history = cnn_model.fit(train_corpus,
                              train_data['enc_label'],
                              epochs = 10,
                              batch_size = 32,
                              verbose = 1,
                              validation_split = 0.1)

Epoch 1/10
148/148 [==============================] - 3s 12ms/step - loss: 1.0979 - accuracy: 0.6065 - val_loss: 0.8355 - val_accuracy: 0.6781
Epoch 2/10
148/148 [==============================] - 1s 9ms/step - loss: 0.5895 - accuracy: 0.7772 - val_loss: 0.7262 - val_accuracy: 0.7600
Epoch 3/10
148/148 [==============================] - 1s 9ms/step - loss: 0.2985 - accuracy: 0.9039 - val_loss: 0.7259 - val_accuracy: 0.7771
Epoch 4/10
148/148 [==============================] - 1s 10ms/step - loss: 0.1003 - accuracy: 0.9795 - val_loss: 0.7275 - val_accuracy: 0.7981
Epoch 5/10
148/148 [==============================] - 1s 9ms/step - loss: 0.0393 - accuracy: 0.9928 - val_loss: 0.7488 - val_accuracy: 0.8038
Epoch 6/10
148/148 [==============================] - 1s 9ms/step - loss: 0.0211 - accuracy: 0.9949 - val_loss: 0.8015 - val_accuracy: 0.8038
Epoch 7/10
148/148 [==============================] - 1s 9ms/step - loss: 0.0161 - accuracy: 0.9968 - val_loss: 0.7962 - val_accuracy: 0.8019
Epoc

In [ ]:
y_pred = np.argmax(cnn_model.predict(test_corpus), axis=-1)

19/19 [==============================] - 0s 4ms/step


In [ ]:
print(classification_report(test_data['enc_label'],y_pred))

              precision    recall  f1-score   support

           0       0.96      0.92      0.94       291
           1       0.84      0.83      0.83       144
           2       0.52      0.75      0.61        55
           3       0.71      0.75      0.73        36
           4       0.75      0.57      0.65        37
           5       0.83      0.71      0.77        21

    accuracy                           0.84       584
   macro avg       0.77      0.75      0.76       584
weighted avg       0.86      0.84      0.85       584



# BiLstm

In [ ]:
keras.backend.clear_session()

max_length = 100
embedding_dim = 100
number_of_classes = 6


# define bilstm model

def lstm():


   ###### BiLSTM Model #######
  bi_text_inputs = Input(shape=(max_length,))
  bi_embedding_layer = Embedding(max_words,embedding_dim)(bi_text_inputs)
  LSTM_Layer_1 = Bidirectional(LSTM(32))(bi_embedding_layer)
  bi_dense_layer_1 = Dense(number_of_classes, activation='softmax')(LSTM_Layer_1)
  bilstm_model = Model(inputs=bi_text_inputs, outputs=bi_dense_layer_1)


  return bilstm_model

# call the model
lstm_model = lstm()

# lstm_model.summary()

In [ ]:
lstm_model.compile(optimizer = Adam(),
                      loss = 'sparse_categorical_crossentropy',
                      metrics =['accuracy'])

In [ ]:
%%time
history = lstm_model.fit(train_corpus,
                              train_data['enc_label'],
                              epochs = 10,
                              batch_size = 32,
                              verbose = 1,
                              validation_split = 0.1)

Epoch 1/10
148/148 [==============================] - 7s 20ms/step - loss: 1.2242 - accuracy: 0.5667 - val_loss: 0.8848 - val_accuracy: 0.6800
Epoch 2/10
148/148 [==============================] - 2s 15ms/step - loss: 0.6288 - accuracy: 0.7696 - val_loss: 0.6879 - val_accuracy: 0.7505
Epoch 3/10
148/148 [==============================] - 2s 15ms/step - loss: 0.3897 - accuracy: 0.8585 - val_loss: 0.6294 - val_accuracy: 0.7790
Epoch 4/10
148/148 [==============================] - 2s 15ms/step - loss: 0.2496 - accuracy: 0.9242 - val_loss: 0.5842 - val_accuracy: 0.8076
Epoch 5/10
148/148 [==============================] - 2s 15ms/step - loss: 0.1371 - accuracy: 0.9767 - val_loss: 0.5445 - val_accuracy: 0.8381
Epoch 6/10
148/148 [==============================] - 2s 15ms/step - loss: 0.0703 - accuracy: 0.9877 - val_loss: 0.5381 - val_accuracy: 0.8419
Epoch 7/10
148/148 [==============================] - 2s 15ms/step - loss: 0.0454 - accuracy: 0.9939 - val_loss: 0.5619 - val_accuracy: 0.8495

In [ ]:
y_pred = np.argmax(lstm_model.predict(test_corpus), axis=-1)

19/19 [==============================] - 1s 6ms/step


In [ ]:
print(classification_report(test_data['enc_label'],y_pred))

              precision    recall  f1-score   support

           0       0.96      0.94      0.95       291
           1       0.82      0.80      0.81       144
           2       0.67      0.64      0.65        55
           3       0.54      0.86      0.67        36
           4       0.89      0.65      0.75        37
           5       0.78      0.86      0.82        21

    accuracy                           0.85       584
   macro avg       0.78      0.79      0.77       584
weighted avg       0.86      0.85      0.85       584



# BiLstm + CNN model

In [ ]:
keras.backend.clear_session()

# define bilstm + Cnn model

def lstm_cnn():


  lc_text_inputs = Input(shape=(max_length,))
  lc_embedding_layer = Embedding(max_words,embedding_dim)(lc_text_inputs)
  LSTM_Layer = Bidirectional(LSTM(32,return_sequences=True))(lc_embedding_layer)
  lc_conv1 = Conv1D(32,2,activation='relu')(LSTM_Layer)
  lc_pool1 = MaxPooling1D(2)(lc_conv1)
  lc_flat = Flatten()(lc_pool1)
  lc_dense_layer_1 = Dense(number_of_classes, activation='softmax')(lc_flat)
  cnn_lstm_model = Model(inputs=lc_text_inputs, outputs=lc_dense_layer_1)


  return cnn_lstm_model

# call the model
lstm_cnn_model = lstm_cnn()

lstm_cnn_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100)]             0         
                                                                 
 embedding (Embedding)       (None, 100, 100)          10000000  
                                                                 
 bidirectional (Bidirectiona  (None, 100, 64)          34048     
 l)                                                              
                                                                 
 conv1d (Conv1D)             (None, 99, 32)            4128      
                                                                 
 max_pooling1d (MaxPooling1D  (None, 49, 32)           0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 1568)              0     

In [ ]:
lstm_cnn_model.compile(optimizer = 'adam',
                      loss = 'sparse_categorical_crossentropy',
                      metrics =['accuracy'])

In [ ]:
%%time
history = lstm_cnn_model.fit(train_corpus,
                              train_data['enc_label'],
                              epochs = 10,
                              batch_size = 32,
                              verbose = 1,
                              validation_split = 0.1)

Epoch 1/10
148/148 [==============================] - 6s 21ms/step - loss: 1.0503 - accuracy: 0.6262 - val_loss: 0.7913 - val_accuracy: 0.7029
Epoch 2/10
148/148 [==============================] - 2s 15ms/step - loss: 0.4774 - accuracy: 0.8158 - val_loss: 0.6013 - val_accuracy: 0.8038
Epoch 3/10
148/148 [==============================] - 2s 16ms/step - loss: 0.2241 - accuracy: 0.9235 - val_loss: 0.6468 - val_accuracy: 0.8286
Epoch 4/10
148/148 [==============================] - 2s 16ms/step - loss: 0.0851 - accuracy: 0.9776 - val_loss: 0.6774 - val_accuracy: 0.8152
Epoch 5/10
148/148 [==============================] - 2s 16ms/step - loss: 0.0364 - accuracy: 0.9915 - val_loss: 0.6918 - val_accuracy: 0.8381
Epoch 6/10
148/148 [==============================] - 2s 16ms/step - loss: 0.0210 - accuracy: 0.9951 - val_loss: 0.7246 - val_accuracy: 0.8514
Epoch 7/10
148/148 [==============================] - 2s 15ms/step - loss: 0.0159 - accuracy: 0.9964 - val_loss: 0.7884 - val_accuracy: 0.8533

In [ ]:
y_pred = np.argmax(lstm_cnn_model.predict(test_corpus), axis=-1)

19/19 [==============================] - 1s 6ms/step


In [ ]:
print(classification_report(test_data['enc_label'],y_pred))

              precision    recall  f1-score   support

           0       0.97      0.96      0.97       291
           1       0.81      0.83      0.82       144
           2       0.69      0.67      0.68        55
           3       0.53      0.78      0.63        36
           4       0.78      0.57      0.66        37
           5       1.00      0.71      0.83        21

    accuracy                           0.86       584
   macro avg       0.80      0.75      0.76       584
weighted avg       0.87      0.86      0.86       584



BILSTM+ATTENtion 01

In [ ]:
from keras.layers import Layer
from tensorflow.python.keras.layers import Concatenate
import keras.backend as K

In [ ]:
max_len = 200
rnn_cell_size = 128

In [ ]:
class Attention(tf.keras.Model):
    def __init__(self, units):
        super(Attention, self).__init__()
        self.W1 = tf.keras.layers.Dense(units)
        self.W2 = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(1)

    def call(self, features, hidden):
        hidden_with_time_axis = tf.expand_dims(hidden, 1)
        score = tf.nn.tanh(self.W1(features) + self.W2(hidden_with_time_axis))
        attention_weights = tf.nn.softmax(self.V(score), axis=1)
        context_vector = attention_weights * features
        context_vector = tf.reduce_sum(context_vector, axis=1)

        return context_vector, attention_weights

In [ ]:
sequence_input = Input(shape=(max_len,), dtype='int32')

embedded_sequences = keras.layers.Embedding(max_words, 128, input_length=max_len)(sequence_input)

In [ ]:
import os
lstm = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM
                                     (rnn_cell_size,
                                      dropout=0.3,
                                      return_sequences=True,
                                      return_state=True,
                                      recurrent_activation='relu',
                                      recurrent_initializer='glorot_uniform'), name="bi_lstm_0")(embedded_sequences)

lstm, forward_h, forward_c, backward_h, backward_c = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(rnn_cell_size,dropout=0.2,return_sequences=True,return_state=True,
      recurrent_activation='relu',
      recurrent_initializer='glorot_uniform'))(lstm)

In [ ]:
state_h = Concatenate()([forward_h, backward_h])
state_c = Concatenate()([forward_c, backward_c])

context_vector, attention_weights = attention(lstm, state_h)

output = keras.layers.Dense(6, activation='softmax')(context_vector)

model = keras.Model(inputs=sequence_input, outputs=output)

# summarize layers
print(model.summary())

AttributeError: ignored

# BiLstm + Attention

In [ ]:
from keras.layers import Layer
import keras.backend as K

In [ ]:
class attention(Layer):
    def __init__(self,**kwargs):
        super(attention,self).__init__(**kwargs)

    def build(self,input_shape):
        self.W=self.add_weight(name="att_weight",shape=(input_shape[-1],1),initializer="normal")
        self.b=self.add_weight(name="att_bias",shape=(input_shape[1],1),initializer="zeros")
        super(attention, self).build(input_shape)

    def call(self,x):
        et=K.squeeze(K.tanh(K.dot(x,self.W)+self.b),axis=-1)
        at=K.softmax(et)
        at=K.expand_dims(at,axis=-1)
        output=x*at
        return K.sum(output,axis=1)

    def compute_output_shape(self,input_shape):
        return (input_shape[0],input_shape[-1])

    def get_config(self):
        return super(attention,self).get_config()

In [ ]:
keras.backend.clear_session()

max_length = 100
embedding_dim = 100
number_of_classes = 6

inputs=Input(shape=(max_length,), name='input')
x=Embedding(max_words,embedding_dim)(inputs)
att_in=Bidirectional(LSTM(256,return_sequences=True,dropout=0.3,recurrent_dropout=0.2))(x)
att_out=attention()(att_in)
outputs=Dense(6,activation='softmax',trainable=True, name="Dense_layer")(att_out)
model=Model(inputs,outputs,name='Text_model')
model.summary()

Model: "Text_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 100)]             0         
                                                                 
 embedding (Embedding)       (None, 100, 100)          10000000  
                                                                 
 bidirectional (Bidirectiona  (None, 100, 512)         731136    
 l)                                                              
                                                                 
 attention (attention)       (None, 512)               612       
                                                                 
 Dense_layer (Dense)         (None, 6)                 3078      
                                                                 
Total params: 10,734,826
Trainable params: 10,734,826
Non-trainable params: 0
____________________________________________

In [ ]:
model.compile(optimizer = Adam(learning_rate=3e-3),
                      loss = 'sparse_categorical_crossentropy',
                      metrics =['accuracy'])

In [ ]:
%%time
history = model.fit(train_corpus,
                              train_data['enc_label'],
                              epochs = 10,
                              batch_size = 32,
                              callbacks=checkpoint_fn('bilstmAttention'),
                              validation_split = 0.1)

Epoch 1/10
148/148 [==============================] - ETA: 0s - loss: 1.0937 - accuracy: 0.6036
Epoch 1: val_accuracy improved from -inf to 0.70095, saving model to /content/drive/MyDrive/New folder/Model/bilstmAttention.h5
148/148 [==============================] - 128s 839ms/step - loss: 1.0937 - accuracy: 0.6036 - val_loss: 0.7622 - val_accuracy: 0.7010
Epoch 2/10
148/148 [==============================] - ETA: 0s - loss: 0.5461 - accuracy: 0.8052
Epoch 2: val_accuracy improved from 0.70095 to 0.78857, saving model to /content/drive/MyDrive/New folder/Model/bilstmAttention.h5
148/148 [==============================] - 122s 827ms/step - loss: 0.5461 - accuracy: 0.8052 - val_loss: 0.5807 - val_accuracy: 0.7886
Epoch 3/10
148/148 [==============================] - ETA: 0s - loss: 0.2761 - accuracy: 0.9144
Epoch 3: val_accuracy improved from 0.78857 to 0.90286, saving model to /content/drive/MyDrive/New folder/Model/bilstmAttention.h5
148/148 [==============================] - 133s 901m

In [ ]:
# from keras.models import load_model
# saved_model = load_model('/content/drive/MyDrive/New folder/Model/bilstmAttention.h5')

In [ ]:
y_pred = np.argmax(model.predict(test_corpus), axis=-1)

19/19 [==============================] - 2s 63ms/step


In [ ]:
print(classification_report(test_data['enc_label'],y_pred))

              precision    recall  f1-score   support

           0       0.96      0.97      0.96       291
           1       0.84      0.83      0.84       144
           2       0.73      0.73      0.73        55
           3       0.66      0.81      0.73        36
           4       0.81      0.68      0.74        37
           5       0.86      0.86      0.86        21

    accuracy                           0.88       584
   macro avg       0.81      0.81      0.81       584
weighted avg       0.88      0.88      0.88       584



# Bert-Embedding

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 57.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 47.8 MB/s eta 0:00:00


In [ ]:
############################## Sentence Encoding for Transformers

def bert_encode(data,maximum_length,tokenizer) :
  input_ids = []
  attention_masks = []


  for i in range(len(data.clean_Text)):
      encoded = tokenizer.encode_plus(

        data.clean_Text[i],
        add_special_tokens=True,
        max_length=maximum_length,
        pad_to_max_length=True,
        truncation=True,
        return_attention_mask=True,
        # return_tensors='pt'

      )

      input_ids.append(encoded['input_ids'])
      attention_masks.append(encoded['attention_mask'])
  return np.array(input_ids),np.array(attention_masks)

In [ ]:
#######################################             Model Defination

#  output[0:, :, :] ----> word embeddings
#  output[:, 0, :]  ----> Sentence Embedding


def create_model(bert_model,max_len):
  input_ids = Input(shape=(max_len,),dtype='int32')
  attention_masks = Input(shape=(max_len,),dtype='int32')
  output = bert_model([input_ids,attention_masks])
  output = output[0]
  # last_hidden_state, pooler_output = output[0]                            ## 0 for distillbert
  #output = Dense(32,activation='relu')(output[:, 0, :])
  #output = Dropout(0.1)(output)

  output1 = Dense(6,activation='softmax')(output[:, 0, :])
  model = Model(inputs = [input_ids,attention_masks],outputs = output1)
  return model

# multilingual-bert

In [ ]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

In [ ]:
vocab_size = tokenizer.vocab_size

print("Vocabulary size:", vocab_size)

Vocabulary size: 119547


In [ ]:
train_input_ids,train_attention_masks = bert_encode(train_data,100,tokenizer)
# valid_input_ids,valid_attention_masks = bert_encode(valid_data,50,tokenizer)
test_input_ids,test_attention_masks = bert_encode(test_data,100,tokenizer)

NameError: ignored

In [ ]:
from transformers import TFBertModel
bert_model = TFBertModel.from_pretrained('bert-base-multilingual-cased')

Some layers from the model checkpoint at bert-base-multilingual-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-multilingual-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [ ]:
with open("vocab.txt", "r", encoding="utf-8") as f:
    vocab_list = f.read().splitlines()

print("Vocabulary size:", len(vocab_list))

FileNotFoundError: ignored

In [ ]:
### Checkpoint
filepath = '/content/drive/MyDrive/New folder/Model/bert_model.h5'
checkpoint = keras.callbacks.ModelCheckpoint(filepath, monitor='val_accuracy', verbose=2, save_best_only=True,
                                             save_weights_only=True, mode='max' )


In [ ]:
keras.backend.clear_session()
# 3e-4, 1e-4, 5e-5, 3e-5,1e-5
model = create_model(bert_model,100)
model.compile(Adam(lr=1e-5), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 100)]        0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 100)]        0           []                               
                                                                                                  
 tf_bert_model (TFBertModel)    TFBaseModelOutputWi  177853440   ['input_1[0][0]',                
                                thPoolingAndCrossAt               'input_2[0][0]']                
                                tentions(last_hidde                                               
                                n_state=(None, 100,                                           

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
model.fit([train_input_ids,train_attention_masks],
                    train_data['enc_label'],
                    validation_split = 0.1,
                    epochs=6,
                    batch_size=16,
                    callbacks = [checkpoint]
                    # class_weight = weight
          )

Epoch 1/6


296/296 [==============================] - ETA: 0s - loss: 0.5687 - accuracy: 0.8035
Epoch 1: val_accuracy improved from -inf to 0.88190, saving model to /content/drive/MyDrive/New folder/Model/bert_model.h5
296/296 [==============================] - 133s 398ms/step - loss: 0.5687 - accuracy: 0.8035 - val_loss: 0.3495 - val_accuracy: 0.8819
Epoch 2/6
296/296 [==============================] - ETA: 0s - loss: 0.2424 - accuracy: 0.9180
Epoch 2: val_accuracy improved from 0.88190 to 0.91619, saving model to /content/drive/MyDrive/New folder/Model/bert_model.h5
296/296 [==============================] - 121s 409ms/step - loss: 0.2424 - accuracy: 0.9180 - val_loss: 0.2875 - val_accuracy: 0.9162
Epoch 3/6
296/296 [==============================] - ETA: 0s - loss: 0.1763 - accuracy: 0.9348
Epoch 3: val_accuracy did not improve from 0.91619
296/296 [==============================] - 114s 385ms/step - loss: 0.1763 - accuracy: 0.9348 - val_loss: 0.2949 - val_accuracy: 0.8952
Epoch 4/6
296/296 [=

In [ ]:
model = create_model(bert_model,100)
# Load the saved model
model.load_weights('/content/drive/MyDrive/New folder/Model/bert_model.h5')
y_pred = np.argmax(model.predict([test_input_ids,test_attention_masks]), axis=-1)
print(classification_report(test_data["enc_label"], y_pred))

19/19 [==============================] - 8s 210ms/step
              precision    recall  f1-score   support

           0       0.96      0.99      0.97       291
           1       0.86      0.87      0.86       144
           2       0.81      0.69      0.75        55
           3       0.72      0.78      0.75        36
           4       0.82      0.73      0.77        37
           5       0.90      0.90      0.90        21

    accuracy                           0.90       584
   macro avg       0.84      0.83      0.83       584
weighted avg       0.90      0.90      0.90       584



In [ ]:
f1_score(test_data["enc_label"], y_pred, average='weighted')

0.8957710864038345

## resnet + bert

In [ ]:
def create_bert_resnet_model ():
  keras.backend.clear_session()
  # create the base pre-trained model
  resnet = ResNet50(weights='imagenet', include_top=False,input_shape=(100, 100, 3))
  for layer in resnet.layers[0:-10]:
      layer.trainable = False
  # add a global spatial average pooling layer
  x = resnet.output
  pool = GlobalAveragePooling2D()(x)
  # let's add a fully-connected layer
  flat = Flatten()(pool)
  # and a logistic layer -- let's say we have 200 classes
  hidden2 = Dense(200, activation='relu')(flat)
  # this is the model we will train
  resnet_img_model = Model(inputs=resnet.input, outputs=hidden2)


  # Text Model
  bert_model = TFBertModel.from_pretrained('bert-base-multilingual-cased')
  input_ids = Input(shape=(100,),dtype='int32')
  attention_masks = Input(shape=(100,),dtype='int32')
  output = bert_model([input_ids,attention_masks])
  output = output[0]
  # last_hidden_state, pooler_output = output[0]                            ## 0 for distillbert
  #output = Dense(32,activation='relu')(output[:, 0, :])
  #output = Dropout(0.1)(output)

  output1 = Dense(200, activation='relu')(output[:, 0, :])
  bert_model = Model(inputs = [input_ids,attention_masks],outputs = output1)

  # Concatenating the output of 2 classifiers
  con_layer = keras.layers.concatenate([resnet_img_model.output, bert_model.output])
  dropout = Dropout(0.2)(con_layer)
  final_dense = Dense(100, activation="relu")(dropout)
  out = Dense(6,activation='softmax')(final_dense)

  #Defining model input and output
  com_model = Model(inputs = [resnet_img_model.input, bert_model.input], outputs=out)

  return com_model

In [ ]:
model = create_bert_resnet_model()

Some layers from the model checkpoint at bert-base-multilingual-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-multilingual-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [ ]:
# 3e-4---->  0.76, 1e-4 ---> 0.72, 5e-5 ---> .8987239486293646, 3e-5---> 0.9316320633014693,1e-5---> .9018863525937759

In [ ]:
model.compile(loss='sparse_categorical_crossentropy',
                      optimizer=Adam(learning_rate=3e-5),
                      metrics = ["accuracy"])

In [ ]:
filepath = f'/content/drive/MyDrive/New folder/Model/bert_resnet.h5'
checkpoint = keras.callbacks.ModelCheckpoint(filepath, monitor='val_accuracy', verbose=2, save_best_only=True,
                                             save_weights_only=True, mode='max' )

In [ ]:
model.fit([train_image,train_input_ids,train_attention_masks],
          train_data['enc_label'],
          batch_size=16,
          epochs=7,
          validation_split=0.10,
          verbose =1,
          callbacks = [checkpoint] )

Epoch 1/7


296/296 [==============================] - ETA: 0s - loss: 0.5584 - accuracy: 0.8111
Epoch 1: val_accuracy improved from -inf to 0.90857, saving model to /content/drive/MyDrive/New folder/Model/bert_resnet.h5
296/296 [==============================] - 181s 518ms/step - loss: 0.5584 - accuracy: 0.8111 - val_loss: 0.2756 - val_accuracy: 0.9086
Epoch 2/7
296/296 [==============================] - ETA: 0s - loss: 0.2088 - accuracy: 0.9348
Epoch 2: val_accuracy did not improve from 0.90857
296/296 [==============================] - 128s 434ms/step - loss: 0.2088 - accuracy: 0.9348 - val_loss: 0.2913 - val_accuracy: 0.9086
Epoch 3/7
296/296 [==============================] - ETA: 0s - loss: 0.1320 - accuracy: 0.9557
Epoch 3: val_accuracy improved from 0.90857 to 0.91238, saving model to /content/drive/MyDrive/New folder/Model/bert_resnet.h5
296/296 [==============================] - 136s 459ms/step - loss: 0.1320 - accuracy: 0.9557 - val_loss: 0.2718 - val_accuracy: 0.9124
Epoch 4/7
296/296 

In [ ]:
model = create_bert_resnet_model()
# Load the saved model
model.load_weights('/content/drive/MyDrive/New folder/Model/bert_resnet.h5')
y_pred = np.argmax(model.predict([test_image,test_input_ids,test_attention_masks]), axis=-1)
print(classification_report(test_data["enc_label"], y_pred))

Some layers from the model checkpoint at bert-base-multilingual-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-multilingual-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


19/19 [==============================] - 9s 262ms/step
              precision    recall  f1-score   support

           0       0.99      0.99      0.99       291
           1       0.88      0.90      0.89       144
           2       0.83      0.82      0.83        55
           3       0.82      0.89      0.85        36
           4       1.00      0.84      0.91        37
           5       0.91      0.95      0.93        21

    accuracy                           0.93       584
   macro avg       0.90      0.90      0.90       584
weighted avg       0.93      0.93      0.93       584



In [ ]:
f1_score(test_data["enc_label"], y_pred, average='weighted')

0.9316320633014693

# English-bert

In [ ]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
train_input_ids,train_attention_masks = bert_encode(train_data,100,tokenizer)
# valid_input_ids,valid_attention_masks = bert_encode(valid_data,50,tokenizer)
test_input_ids,test_attention_masks = bert_encode(test_data,100,tokenizer)

/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2336: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [ ]:
from transformers import TFBertModel
bert_model = TFBertModel.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/536M [00:00<?, ?B/s]

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [ ]:
### Checkpoint
filepath = '/content/drive/MyDrive/New folder/Model/bert_eng_model.h5'
checkpoint = keras.callbacks.ModelCheckpoint(filepath, monitor='val_accuracy', verbose=2, save_best_only=True,
                                             save_weights_only=True, mode='max' )


In [ ]:
keras.backend.clear_session()
# 3e-4, 1e-4, 5e-5, 3e-5,1e-5
model = create_model(bert_model,100)
model.compile(Adam(lr=1e-5), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 100)]        0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 100)]        0           []                               
                                                                                                  
 tf_bert_model (TFBertModel)    TFBaseModelOutputWi  109482240   ['input_1[0][0]',                
                                thPoolingAndCrossAt               'input_2[0][0]']                
                                tentions(last_hidde                                               
                                n_state=(None, 100,                                           

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
model.fit([train_input_ids,train_attention_masks],
                    train_data['enc_label'],
                    validation_split = 0.1,
                    epochs=6,
                    batch_size=16,
                    callbacks = [checkpoint]
                    # class_weight = weight
          )

Epoch 1/6


296/296 [==============================] - ETA: 0s - loss: 0.5891 - accuracy: 0.7999
Epoch 1: val_accuracy improved from -inf to 0.89905, saving model to /content/drive/MyDrive/New folder/Model/bert_eng_model.h5
296/296 [==============================] - 125s 374ms/step - loss: 0.5891 - accuracy: 0.7999 - val_loss: 0.2929 - val_accuracy: 0.8990
Epoch 2/6
296/296 [==============================] - ETA: 0s - loss: 0.2365 - accuracy: 0.9174
Epoch 2: val_accuracy did not improve from 0.89905
296/296 [==============================] - 106s 357ms/step - loss: 0.2365 - accuracy: 0.9174 - val_loss: 0.2674 - val_accuracy: 0.8990
Epoch 3/6
296/296 [==============================] - ETA: 0s - loss: 0.1671 - accuracy: 0.9416
Epoch 3: val_accuracy improved from 0.89905 to 0.91619, saving model to /content/drive/MyDrive/New folder/Model/bert_eng_model.h5
296/296 [==============================] - 111s 375ms/step - loss: 0.1671 - accuracy: 0.9416 - val_loss: 0.2806 - val_accuracy: 0.9162
Epoch 4/6
29

In [ ]:
model = create_model(bert_model,100)
# Load the saved model
model.load_weights('/content/drive/MyDrive/New folder/Model/bert_eng_model.h5')
y_pred = np.argmax(model.predict([test_input_ids,test_attention_masks]), axis=-1)
print(classification_report(test_data["enc_label"], y_pred))

19/19 [==============================] - 6s 204ms/step
              precision    recall  f1-score   support

           0       0.97      0.98      0.97       291
           1       0.89      0.81      0.85       144
           2       0.81      0.71      0.76        55
           3       0.56      0.83      0.67        36
           4       0.76      0.76      0.76        37
           5       0.87      0.95      0.91        21

    accuracy                           0.89       584
   macro avg       0.81      0.84      0.82       584
weighted avg       0.89      0.89      0.89       584



## eng_bert + resnet

In [ ]:
def create_engbert_resnet_model ():
  keras.backend.clear_session()
  # create the base pre-trained model
  resnet = ResNet50(weights='imagenet', include_top=False,input_shape=(100, 100, 3))
  for layer in resnet.layers[0:-10]:
      layer.trainable = False
  # add a global spatial average pooling layer
  x = resnet.output
  pool = GlobalAveragePooling2D()(x)
  # let's add a fully-connected layer
  flat = Flatten()(pool)
  # and a logistic layer -- let's say we have 200 classes
  hidden2 = Dense(200, activation='relu')(flat)
  # this is the model we will train
  resnet_img_model = Model(inputs=resnet.input, outputs=hidden2)


  # Text Model
  bert_model = TFBertModel.from_pretrained('bert-base-uncased')
  input_ids = Input(shape=(100,),dtype='int32')
  attention_masks = Input(shape=(100,),dtype='int32')
  output = bert_model([input_ids,attention_masks])
  output = output[0]
  # last_hidden_state, pooler_output = output[0]                            ## 0 for distillbert
  #output = Dense(32,activation='relu')(output[:, 0, :])
  #output = Dropout(0.1)(output)

  output1 = Dense(200, activation='relu')(output[:, 0, :])
  bert_model = Model(inputs = [input_ids,attention_masks],outputs = output1)

  # Concatenating the output of 2 classifiers
  con_layer = keras.layers.concatenate([resnet_img_model.output, bert_model.output])
  dropout = Dropout(0.2)(con_layer)
  final_dense = Dense(100, activation="relu")(dropout)
  out = Dense(6,activation='softmax')(final_dense)

  #Defining model input and output
  com_model = Model(inputs = [resnet_img_model.input, bert_model.input], outputs=out)

  return com_model

In [ ]:
model = create_engbert_resnet_model()

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [ ]:
# 3e-4---->  0.76, 1e-4 ---> 0.72, 5e-5 ---> .8987239486293646, 3e-5---> 0.9316320633014693,1e-5---> .9018863525937759

In [ ]:
model.compile(loss='sparse_categorical_crossentropy',
                      optimizer=Adam(learning_rate=3e-5),
                      metrics = ["accuracy"])

In [ ]:
filepath = f'/content/drive/MyDrive/New folder/Model/engbert_resnet.h5'
checkpoint = keras.callbacks.ModelCheckpoint(filepath, monitor='val_accuracy', verbose=2, save_best_only=True,
                                             save_weights_only=True, mode='max' )

In [ ]:
model.fit([train_image,train_input_ids,train_attention_masks],
          train_data['enc_label'],
          batch_size=16,
          epochs=7,
          validation_split=0.10,
          verbose =1,
          callbacks = [checkpoint] )

Epoch 1/7


296/296 [==============================] - ETA: 0s - loss: 0.5076 - accuracy: 0.8266
Epoch 1: val_accuracy improved from -inf to 0.89714, saving model to /content/drive/MyDrive/New folder/Model/engbert_resnet.h5
296/296 [==============================] - 149s 417ms/step - loss: 0.5076 - accuracy: 0.8266 - val_loss: 0.2673 - val_accuracy: 0.8971
Epoch 2/7
296/296 [==============================] - ETA: 0s - loss: 0.1752 - accuracy: 0.9401
Epoch 2: val_accuracy improved from 0.89714 to 0.91429, saving model to /content/drive/MyDrive/New folder/Model/engbert_resnet.h5
296/296 [==============================] - 122s 410ms/step - loss: 0.1752 - accuracy: 0.9401 - val_loss: 0.2768 - val_accuracy: 0.9143
Epoch 3/7
296/296 [==============================] - ETA: 0s - loss: 0.1017 - accuracy: 0.9651
Epoch 3: val_accuracy improved from 0.91429 to 0.92190, saving model to /content/drive/MyDrive/New folder/Model/engbert_resnet.h5
296/296 [==============================] - 121s 409ms/step - loss: 0

In [ ]:
model = create_engbert_resnet_model()
# Load the saved model
model.load_weights('/content/drive/MyDrive/New folder/Model/engbert_resnet.h5')
y_pred = np.argmax(model.predict([test_image,test_input_ids,test_attention_masks]), axis=-1)
print(classification_report(test_data["enc_label"], y_pred))

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


19/19 [==============================] - 8s 253ms/step
              precision    recall  f1-score   support

           0       0.97      0.98      0.98       291
           1       0.89      0.87      0.88       144
           2       0.86      0.76      0.81        55
           3       0.72      0.86      0.78        36
           4       0.89      0.89      0.89        37
           5       0.95      0.90      0.93        21

    accuracy                           0.92       584
   macro avg       0.88      0.88      0.88       584
weighted avg       0.92      0.92      0.92       584



In [ ]:
f1_score(test_data["enc_label"], y_pred, average='weighted')

0.9177890050020595

# Tweet-bert

In [ ]:

from transformers import AutoModel, AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base", use_fast=False)


emoji is not installed, thus not converting emoticons or emojis into text. Install emoji: pip3 install emoji==0.6.0
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
train_input_ids,train_attention_masks = bert_encode(train_data,100,tokenizer)
# valid_input_ids,valid_attention_masks = bert_encode(valid_data,50,tokenizer)
test_input_ids,test_attention_masks = bert_encode(test_data,100,tokenizer)

/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2336: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [ ]:

from transformers import TFAutoModel
bertweet = TFAutoModel.from_pretrained("vinai/bertweet-base")
# bertweet = AutoModel.from_pretrained("vinai/bertweet-base")
# # bert_model = TFBertModel.from_pretrained('vinai/bertweet-base')

Some layers from the model checkpoint at vinai/bertweet-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at vinai/bertweet-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


In [ ]:
### Checkpoint
filepath = '/content/drive/MyDrive/New folder/Model/bert_tweet_model.h5'
checkpoint = keras.callbacks.ModelCheckpoint(filepath, monitor='val_accuracy', verbose=2, save_best_only=True,
                                             save_weights_only=True, mode='max' )


In [ ]:
keras.backend.clear_session()
# 3e-4, 1e-4, 5e-5, 3e-5,1e-5
model = create_model(bertweet,100)
model.compile(Adam(lr=1e-5), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 100)]        0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 100)]        0           []                               
                                                                                                  
 tf_roberta_model (TFRobertaMod  TFBaseModelOutputWi  134899968  ['input_1[0][0]',                
 el)                            thPoolingAndCrossAt               'input_2[0][0]']                
                                tentions(last_hidde                                               
                                n_state=(None, 100,                                           

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
model.fit([train_input_ids,train_attention_masks],
                    train_data['enc_label'],
                    validation_split = 0.1,
                    epochs=6,
                    batch_size=16,
                    callbacks = [checkpoint]
                    # class_weight = weight
          )

Epoch 1/6


296/296 [==============================] - ETA: 0s - loss: 0.5985 - accuracy: 0.7999
Epoch 1: val_accuracy improved from -inf to 0.88381, saving model to /content/drive/MyDrive/New folder/Model/bert_tweet_model.h5
296/296 [==============================] - 126s 373ms/step - loss: 0.5985 - accuracy: 0.7999 - val_loss: 0.3268 - val_accuracy: 0.8838
Epoch 2/6
296/296 [==============================] - ETA: 0s - loss: 0.2504 - accuracy: 0.9195
Epoch 2: val_accuracy improved from 0.88381 to 0.89905, saving model to /content/drive/MyDrive/New folder/Model/bert_tweet_model.h5
296/296 [==============================] - 111s 375ms/step - loss: 0.2504 - accuracy: 0.9195 - val_loss: 0.2943 - val_accuracy: 0.8990
Epoch 3/6
296/296 [==============================] - ETA: 0s - loss: 0.1971 - accuracy: 0.9348
Epoch 3: val_accuracy improved from 0.89905 to 0.90476, saving model to /content/drive/MyDrive/New folder/Model/bert_tweet_model.h5
296/296 [==============================] - 111s 375ms/step - l

In [ ]:
model = create_model(bertweet,100)
# Load the saved model
model.load_weights('/content/drive/MyDrive/New folder/Model/bert_tweet_model.h5')
y_pred = np.argmax(model.predict([test_input_ids,test_attention_masks]), axis=-1)
print(classification_report(test_data["enc_label"], y_pred))

19/19 [==============================] - 7s 200ms/step
              precision    recall  f1-score   support

           0       0.98      0.98      0.98       291
           1       0.90      0.85      0.87       144
           2       0.77      0.73      0.75        55
           3       0.65      0.83      0.73        36
           4       0.82      0.84      0.83        37
           5       0.91      0.95      0.93        21

    accuracy                           0.90       584
   macro avg       0.84      0.86      0.85       584
weighted avg       0.91      0.90      0.90       584



# Multimodal

In [ ]:
from keras.layers import concatenate
from tensorflow.keras.layers import Concatenate, Dense, Input, LSTM, Embedding, Dropout, Activation, GRU, Flatten

In [ ]:
def create_lstm_attention_resnet_model ():
  keras.backend.clear_session()
  # create the base pre-trained model
  resnet = ResNet50(weights='imagenet', include_top=False,input_shape=(100, 100, 3))
  for layer in resnet.layers[0:-10]:
      layer.trainable = False
  # add a global spatial average pooling layer
  x = resnet.output
  pool = GlobalAveragePooling2D()(x)
  # let's add a fully-connected layer
  flat = Flatten()(pool)
  # and a logistic layer -- let's say we have 200 classes
  hidden2 = Dense(200, activation='relu')(flat)
  # this is the model we will train
  resnet_img_model = Model(inputs=resnet.input, outputs=hidden2)


  # Text Model

  class Attention(tf.keras.Model):
      def __init__(self, units):
          super(Attention, self).__init__()
          self.W1 = tf.keras.layers.Dense(units)
          self.W2 = tf.keras.layers.Dense(units)
          self.V = tf.keras.layers.Dense(1)

      def call(self, features, hidden):
          # hidden shape == (batch_size, hidden size)
          # hidden_with_time_axis shape == (batch_size, 1, hidden size)
          # we are doing this to perform addition to calculate the score
          hidden_with_time_axis = tf.expand_dims(hidden, 1)

          # score shape == (batch_size, max_length, 1)
          # we get 1 at the last axis because we are applying score to self.V
          # the shape of the tensor before applying self.V is (batch_size, max_length, units)
          score = tf.nn.tanh(
              self.W1(features) + self.W2(hidden_with_time_axis))
          # attention_weights shape == (batch_size, max_length, 1)
          attention_weights = tf.nn.softmax(self.V(score), axis=1)

          # context_vector shape after sum == (batch_size, hidden_size)
          context_vector = attention_weights * features
          context_vector = tf.reduce_sum(context_vector, axis=1)
          return context_vector, attention_weights

  sequence_input = Input(shape=(100,), dtype="int32")
  embedded_sequences = Embedding(vocab_size, 100)(sequence_input)
  lstm = Bidirectional(LSTM(128, return_sequences = True,dropout=0.01), name="bi_lstm_0")(embedded_sequences)
  # Getting our LSTM outputs
  (lstm, forward_h, forward_c, backward_h, backward_c) = Bidirectional(LSTM(128, return_sequences=True, return_state=True), name="bi_lstm_1")(lstm)

  state_h = Concatenate()([forward_h, backward_h])
  state_c = Concatenate()([forward_c, backward_c])
  context_vector, attention_weights = Attention(20)(lstm, state_h)
  dense1 = Dense(200, activation="relu")(context_vector)


  lstm_attention_model = Model(inputs=sequence_input, outputs=dense1)

  # Concatenating the output of 2 classifiers
  con_layer = keras.layers.concatenate([resnet_img_model.output, lstm_attention_model.output])
  dropout = Dropout(0.2)(con_layer)
  final_dense = Dense(100, activation="relu")(dropout)
  out = Dense(6,activation='softmax')(final_dense)

  #Defining model input and output
  com_model = Model(inputs = [resnet_img_model.input, lstm_attention_model.input], outputs=out)

  return com_model

In [ ]:
model = create_lstm_attention_resnet_model()

In [ ]:
model.compile(loss='sparse_categorical_crossentropy',
                      optimizer=Adam(),
                      metrics = ["accuracy"])

In [ ]:
filepath = f'/content/drive/MyDrive/New folder/Model/Attn_multi.h5'
checkpoint = keras.callbacks.ModelCheckpoint(filepath, monitor='val_accuracy', verbose=2, save_best_only=True,
                                             save_weights_only=True, mode='max' )

In [ ]:
model.fit([train_image,train_corpus],
          train_data['enc_label'],
          batch_size=32,
          epochs=50,
          validation_split=0.10,
          verbose =1,
          callbacks = [checkpoint] )

Epoch 1/50
148/148 [==============================] - ETA: 0s - loss: 0.7541 - accuracy: 0.7325
Epoch 1: val_accuracy improved from -inf to 0.79238, saving model to /content/drive/MyDrive/New folder/Model/Attn_multi.h5
148/148 [==============================] - 22s 93ms/step - loss: 0.7541 - accuracy: 0.7325 - val_loss: 0.7999 - val_accuracy: 0.7924
Epoch 2/50
148/148 [==============================] - ETA: 0s - loss: 0.3102 - accuracy: 0.8988
Epoch 2: val_accuracy improved from 0.79238 to 0.82286, saving model to /content/drive/MyDrive/New folder/Model/Attn_multi.h5
148/148 [==============================] - 10s 69ms/step - loss: 0.3102 - accuracy: 0.8988 - val_loss: 0.6268 - val_accuracy: 0.8229
Epoch 3/50
148/148 [==============================] - ETA: 0s - loss: 0.1401 - accuracy: 0.9551
Epoch 3: val_accuracy improved from 0.82286 to 0.86095, saving model to /content/drive/MyDrive/New folder/Model/Attn_multi.h5
148/148 [==============================] - 10s 70ms/step - loss: 0.1401

KeyboardInterrupt: ignored

In [ ]:
model = create_lstm_attention_resnet_model()
# Load the saved model
model.load_weights('/content/drive/MyDrive/New folder/Model/Attn_multi.h5')
y_pred = np.argmax(model.predict([test_image,test_corpus]), axis=-1)
print(classification_report(test_data["enc_label"], y_pred))

19/19 [==============================] - 3s 59ms/step
              precision    recall  f1-score   support

           0       0.98      0.96      0.97       291
           1       0.83      0.90      0.87       144
           2       0.93      0.69      0.79        55
           3       0.72      0.81      0.76        36
           4       0.82      0.89      0.86        37
           5       0.82      0.86      0.84        21

    accuracy                           0.90       584
   macro avg       0.85      0.85      0.85       584
weighted avg       0.91      0.90      0.90       584



In [ ]:
f1_score(test_data["enc_label"], y_pred, average='weighted')

0.9041570181626415

# Inception + BilstmAttention

In [ ]:
keras.backend.clear_session()

max_length = 100
embedding_dim = 100
number_of_classes = 6

inputs=Input(shape=(max_length,), name='input')
x=Embedding(max_words,embedding_dim)(inputs)
att_in=Bidirectional(LSTM(32,return_sequences=True,dropout=0.3,recurrent_dropout=0.2))(x)
att_out=attention()(att_in)
branch_1 = Dense(512, activation='relu')(att_out)

# Image input branch - a pre-trained Inception module followed by an added fully connected layer
base_model = InceptionV3(weights='imagenet', include_top=False)
# Freeze Inception's weights - we don't want to train these
for layer in base_model.layers:
    layer.trainable = False

# add a fully connected layer after Inception - we do want to train these
branch_2 = base_model.output
branch_2 = GlobalAveragePooling2D()(branch_2)
branch_2 = Dense(1024, activation='relu')(branch_2)

# merge the text input branch and the image input branch and add another fully connected layer
joint = concatenate([branch_1, branch_2])
joint = Dense(512, activation='relu')(joint)
joint = Dropout(0.5)(joint)
predictions = Dense(6, activation='softmax')(joint)

full_model = Model(inputs=[base_model.input,inputs], outputs=[predictions])

full_model.compile(loss='sparse_categorical_crossentropy',
                   optimizer='rmsprop',
                   metrics=['accuracy'])


In [ ]:

history = full_model.fit([train_image, train_corpus], train_data['enc_label'],
                         epochs=10, batch_size=32,
                         verbose=1, validation_split=0.1, shuffle=True)

Epoch 1/10
148/148 [==============================] - 130s 825ms/step - loss: 15.1645 - accuracy: 0.4884 - val_loss: 2.0563 - val_accuracy: 0.6838
Epoch 2/10
148/148 [==============================] - 121s 815ms/step - loss: 1.1835 - accuracy: 0.6815 - val_loss: 0.8587 - val_accuracy: 0.6990
Epoch 3/10
148/148 [==============================] - 118s 798ms/step - loss: 0.8220 - accuracy: 0.7313 - val_loss: 0.9588 - val_accuracy: 0.6419
Epoch 4/10
148/148 [==============================] - 121s 818ms/step - loss: 0.6938 - accuracy: 0.7637 - val_loss: 0.9106 - val_accuracy: 0.6514
Epoch 5/10
148/148 [==============================] - 119s 802ms/step - loss: 0.6219 - accuracy: 0.7781 - val_loss: 1.6691 - val_accuracy: 0.7143
Epoch 6/10
148/148 [==============================] - 119s 805ms/step - loss: 0.5639 - accuracy: 0.8030 - val_loss: 0.8522 - val_accuracy: 0.7467
Epoch 7/10
148/148 [==============================] - 118s 796ms/step - loss: 0.5323 - accuracy: 0.8073 - val_loss: 0.9106 

In [ ]:
y_pred = np.argmax(model.predict(test_corpus), axis=-1)

19/19 [==============================] - 1s 53ms/step


In [ ]:
print(classification_report(test_data['enc_label'],y_pred))

              precision    recall  f1-score   support

           0       0.96      0.98      0.97       291
           1       0.89      0.76      0.82       144
           2       0.75      0.75      0.75        55
           3       0.68      0.83      0.75        36
           4       0.69      0.73      0.71        37
           5       0.77      0.95      0.85        21

    accuracy                           0.88       584
   macro avg       0.79      0.83      0.81       584
weighted avg       0.88      0.88      0.88       584



In [ ]:
f1_score(test_data["enc_label"], y_pred, average='weighted')

0.8766296049066437

# Resnet50 + BilstmAttention

In [ ]:
keras.backend.clear_session()

max_length = 100
embedding_dim = 100
number_of_classes = 6

inputs=Input(shape=(max_length,), name='input')
x=Embedding(max_words,embedding_dim)(inputs)
att_in=Bidirectional(LSTM(256,return_sequences=True,dropout=0.3,recurrent_dropout=0.2))(x)
att_out=attention()(att_in)
branch_1 = Dense(256, activation='relu')(att_out)

# Image input branch - a pre-trained Inception module followed by an added fully connected layer
base_model = ResNet50(weights='imagenet', include_top=False)
# Freeze Inception's weights - we don't want to train these
for layer in base_model.layers:
    layer.trainable = False

# add a fully connected layer after Inception - we do want to train these
branch_2 = base_model.output
branch_2 = GlobalAveragePooling2D()(branch_2)
branch_2 = Dense(512, activation='relu')(branch_2)

# merge the text input branch and the image input branch and add another fully connected layer
joint = concatenate([branch_1, branch_2])
joint = Dense(100, activation='relu')(joint)
# joint = Dropout(0.5)(joint)
predictions = Dense(6, activation='softmax')(joint)

full_model = Model(inputs=[base_model.input,inputs], outputs=[predictions])

full_model.compile(loss='sparse_categorical_crossentropy',
                   optimizer=Adam(),
                   metrics=['accuracy'])


In [ ]:
history = full_model.fit([train_image, train_corpus], train_data['enc_label'],
                         epochs=10, batch_size=32,
                         verbose=1, validation_split=0.1, callbacks=checkpoint_fn('ResBiLstmAttention'),shuffle=True)

Epoch 1/10
148/148 [==============================] - ETA: 0s - loss: 0.8883 - accuracy: 0.6872
Epoch 1: val_accuracy improved from -inf to 0.73905, saving model to /content/drive/MyDrive/New folder/Model/ResBiLstmAttention.h5
148/148 [==============================] - 153s 933ms/step - loss: 0.8883 - accuracy: 0.6872 - val_loss: 0.7871 - val_accuracy: 0.7390
Epoch 2/10
148/148 [==============================] - ETA: 0s - loss: 0.4523 - accuracy: 0.8386
Epoch 2: val_accuracy improved from 0.73905 to 0.77524, saving model to /content/drive/MyDrive/New folder/Model/ResBiLstmAttention.h5
148/148 [==============================] - 133s 899ms/step - loss: 0.4523 - accuracy: 0.8386 - val_loss: 0.6807 - val_accuracy: 0.7752
Epoch 3/10
148/148 [==============================] - ETA: 0s - loss: 0.2665 - accuracy: 0.9039
Epoch 3: val_accuracy improved from 0.77524 to 0.79048, saving model to /content/drive/MyDrive/New folder/Model/ResBiLstmAttention.h5
148/148 [==============================] - 

In [ ]:
y_pred = np.argmax(full_model.predict([test_image,test_corpus]), axis=-1)

19/19 [==============================] - 3s 95ms/step


In [ ]:
print(classification_report(test_data["enc_label"], y_pred))

              precision    recall  f1-score   support

           0       0.97      0.97      0.97       291
           1       0.85      0.81      0.83       144
           2       0.66      0.75      0.70        55
           3       0.73      0.75      0.74        36
           4       0.79      0.81      0.80        37
           5       0.88      0.71      0.79        21

    accuracy                           0.88       584
   macro avg       0.81      0.80      0.81       584
weighted avg       0.88      0.88      0.88       584



In [ ]:
f1_score(test_data["enc_label"], y_pred, average='weighted')

0.8790411547323996